##  高斯混合模型

<font color='red' size=4>定义:</font>    
&emsp;&emsp;高斯混合模型是指具有如下形式的概率分布模型:      
$$  P(\mathbf{y}|\theta) = \sum_{k=1}^K \alpha_k N(\mathbf{y}|\boldsymbol{\mu}_k,\Sigma_k)  $$        
该分布共由$ K $个混合成分组成,每个混合成分对应一个高斯分布.其中,$ \boldsymbol{\mu}_k $与$ \Sigma_k $是第$k$个高斯混合成分的参数,
而$ \alpha_k >0$为相应的"混合系数"(mixture coefficient),$ \sum_{k=1}^K \alpha_k=1 $;      


## 高斯混合模型参数估计的EM算法          

&emsp;&emsp;假设观测数据$ \mathbf{y}_1,\mathbf{y}_2,\cdots,\mathbf{y}_N $由高斯混合模型生成,     
$$ P(\mathbf{y}|\theta) = \sum_{k=1}^K \alpha_k N(\mathbf{y}|\boldsymbol{\mu}_k,\Sigma_k) \tag{1} $$         
其中,$\theta = (\alpha_1,\cdots,\alpha_K;\boldsymbol{\mu}_1,\cdots,\boldsymbol{\mu}_K;\Sigma_1,\cdots,\Sigma_K)$.
下面用EM算法估计高斯混合模型的参数$ \theta $.            


### 1. 明确隐变量,写出完全数据的对数似然函数

&emsp;&emsp;可以设想观测数据$ \mathbf{y}_j $,$ j=1,2,\cdots,N $,是这样产生的:首先依概率$ \alpha_k $选择第$ k $个高斯
分布分模型$ N(\mathbf{y}|\boldsymbol{\mu}_k,\Sigma_k) $,然后依第$ k $个分模型的概率分布$ N(\mathbf{y}|\boldsymbol{\mu}_k,\Sigma_k) $生成观测数据$ \mathbf{y}_j $.
这时观测数据$ \mathbf{y}_j $,$ j=1,2,\cdots,N $,是已知的;反映观测数据$ \mathbf{y}_j $来自第$ k $个分模型的数据是未知的,$ k=1,2,\cdots,K $,以隐变量$ \gamma_{jk} $表示,其定义如下:            

$$
\gamma_{jk}=\begin{cases}
		1, &  \text{第}j\text{个观测来自第}k\text{个分模型} \\
        0, &  \text{否则} 
     \end{cases}
$$    

$$ j=1,2,\cdots,N; \quad k=1,2,\cdots,K  $$

$\gamma_{jk}$是0-1随机变量.

&emsp;&emsp;有了观测数据$ \mathbf{y}_j $及未观测数据$ \gamma_{jk} $,那么完全数据是            
$$ (\mathbf{y}_j,\gamma_{j1},\gamma_{j2},\cdots,\gamma_{jK}),\quad j=1,2,\cdots,N $$              
&emsp;&emsp;于是,可以写出完全数据的似然函数:         

$$
\begin{aligned}
P(\mathbf{y},\gamma|\theta) &= \prod_{j=1}^{N} P(\mathbf{y}_j,\gamma_{j1},\gamma_{j2},\cdots,\gamma_{jK} | \theta) \\
				   &= \prod_{j=1}^N P(\mathbf{y}_j|\gamma_{j1},\gamma_{j2},\cdots,\gamma_{jK},\theta) P(\gamma_{j1},\gamma_{j2},\cdots,\gamma_{jK} | \theta) \\ 
				   &= \prod_{j=1}^N \prod_{k=1}^K [ \alpha_k N(\mathbf{y}_j|\boldsymbol{\mu}_k,\Sigma_k) ]^{\gamma_{jk}} \\
		           &= \prod_{k=1}^K \prod_{j=1}^N [ \alpha_k N(\mathbf{y}_j|\boldsymbol{\mu}_k,\Sigma_k) ]^{\gamma_{jk}} \\
		           &= \prod_{k=1}^K \alpha_k^{n_k} \prod_{j=1}^N N(\mathbf{y}_j|\boldsymbol{\mu}_k,\Sigma_k)^{\gamma_{jk}} 
\end{aligned} 
$$      

式中;$ n_k = \sum_{j=1}^N \gamma_{jk}, \quad \sum_{k=1}^K n_k = N $            
&emsp;&emsp;那么完全数据的对数似然函数为          
$$\log P(\mathbf{y},\gamma|\theta) = \sum_{k=1}^K \left\{ n_k \log \alpha_k + \sum_{j=1}^N \gamma_{jk} \log \left( N(\mathbf{y}_j|\mu_k,\Sigma_k) \right)  \right\} $$     


### EM算法的E步,确定Q函数     

$$
\begin{aligned}
Q(\theta, \theta^{(i)}) &= E_{\gamma |y, \theta^{(i)}}[ \log P(\mathbf{y},\gamma|\theta)] \\
						&= E_{\gamma |y, \theta^{(i)}} \left\{ \sum_{k=1}^K \left\{ n_k \log \alpha_k + \sum_{j=1}^N \gamma_{jk} \log \left( N(\mathbf{y}_j|\boldsymbol{\mu}_k,\Sigma_k) \right)  \right\} \right\} \\
						&=  \sum_{k=1}^K \left\{ \sum_{j=1}^N \left(E( \gamma_{rk} |\mathbf{y}, \theta^{(i)} ) \right) \log \alpha_k + \sum_{j=1}^N \left(E( \gamma_{rk} |\mathbf{y}, \theta^{(i)} ) \right) \log \left( N(\mathbf{y}_j|\boldsymbol{\mu}_k,\Sigma_k) \right)  \right\}  \\
\end{aligned}
\tag{2}
$$

&emsp;&emsp;这里需要计算$ E( \gamma_{rk} |\mathbf{y}, \theta^{(i)}) $,记为$ \hat{\gamma}_{jk} $           

$$
\begin{aligned}
\hat{\gamma}_{jk} &= E( \gamma_{rk} |\mathbf{y}, \theta^{(i)} )  \\
				  &= P(\gamma_{jk}=1 |y, \theta^{(i)})  \\
				  &= \frac{P(\gamma_{jk}=1, \mathbf{y}| \theta^{(i)})}{ P(y|\theta^{(i)}) } \quad \text{贝叶斯公式}  \\
				  &= \frac{P(\gamma_{jk}=1, \mathbf{y}_j| \theta^{(i)}) }{ \sum_{k=1}^K P(\gamma_{jk}=1,\mathbf{y}_j | \theta^{(i)})} \quad \text{全概率公式} \quad \gamma_{jk} \text{只与} y_j \text{有关} \\
				  &= \frac{P( \mathbf{y}_j | \gamma_{jk}=1, \theta^{(i)}) P(\gamma_{rk}=1|\theta^{(i)}) }{ \sum_{k=1}^K P(\mathbf{y}_j|\gamma_{jk}=1, \theta^{(i)}) P(\gamma_{rk}=1|\theta^{(i)})} \quad \text{乘法公式} \\ 
				  &= \frac{\alpha_k  N(\mathbf{y}_j|\boldsymbol{\mu}_k^{(i)},\Sigma_k^{(i)})}{\sum_{k=1}^k \alpha_k   N(\mathbf{y}_j|\boldsymbol{\mu}_k^{(i)},\Sigma_k^{(i)})}, \quad j=1,2,\cdots,N; \quad k=1,2,\cdots,K \\
\end{aligned}
\tag{3}
$$      

$ \hat{\gamma}_{jk} $是在当前模型参数下第$j$个观测数据来自第$k$个分模型的概率,称为分模型$k$对观测数据$\mathbf{y}_j$的响应度.        
&emsp;&emsp;易知,$ \sum_{j=1}^N E\left( \gamma_{rk} |\mathbf{y}, \theta^{(i)} \right)= E_{\gamma_{rk} |\mathbf{y}, \theta^{(i)}} \left( \sum_{j=1}^N r_{jk} \right) = E_{\gamma_{rk} |\mathbf{y}, \theta^{(i)}} n_k = n_k  $         
故将$ \hat{\gamma}_{jk} = E( \gamma_{rk} |\mathbf{y}, \theta^{(i)} )$及其$ n_k = \sum_{j=1}^N E\left( \gamma_{rk} |\mathbf{y}, \theta^{(i)} \right)  $代入式(2),即得      
$$ Q(\theta, \theta^{(i)}) = \sum_{k=1}^K \left\{ n_k \log \alpha_k + \sum_{j=1}^N \hat{\gamma}_{jk} \log \left( N(\mathbf{y}_j|\boldsymbol{\mu}_k,\Sigma_k) \right)  \right\} \tag{4} $$        


### 确定EM算法的M步

&emsp;&emsp;迭代的M步是求函数$ Q(\theta, \theta^{(i)}) $对$ \theta $的极大值,即求新一轮迭代的模型参数:        
$$ \theta^{(i+1)} = \mathrm{arg} \max_{\theta} Q(\theta, \theta^{(i)}) $$        
用$ \boldsymbol{\mu}_k', \Sigma_k'$及$\alpha_k' $,$ k=1,2,\cdots,K$分别表示$\theta^{(i+1)} $的各参数.
求$\boldsymbol{\mu}_k', \Sigma_k'$只需将式(3)分别对$\boldsymbol{\mu}_k', \Sigma_k'$求偏导并令其为0,
即可解得;而$\alpha_k'$是在$ \sum_{k=1}^K \alpha_k=1 $条件下求偏导并令其为0得到的,结果如下:                  
已知           
$$ N(\mathbf{y}_j|\boldsymbol{\mu}_k,\Sigma_k) = \frac{1}{(2\pi)^{\frac{n}{2}}|\Sigma|^{\frac{1}{2}}} e^{-\frac{1}{2} (\mathbf{x} - \boldsymbol{\mu})^T \Sigma^{-1}(\mathbf{x} - \boldsymbol{\mu})} $$    
于是          

$$
\begin{aligned}
\frac{\partial Q(\theta, \theta^{(i)})}{\partial  \boldsymbol{\mu}_k} &= \frac{\partial \left[ \sum_{j=1}^N \hat{\gamma}_{jk} \log \left( N(\mathbf{y}_j|\boldsymbol{\mu}_k,\Sigma_k) \right) \right]}{\partial \boldsymbol{\mu}_k}  \\
				&=\frac{ \partial \left[ \sum_{j=1}^N \hat{\gamma}_{jk} \left( -\frac{1}{2} (\mathbf{y}_j - \boldsymbol{\mu}_k)^T \Sigma_k^{-1}(\mathbf{y}_j - \boldsymbol{\mu}_k) \right) \right] }{\partial \boldsymbol{\mu}_k} \\
				&= -\frac{1}{2} \sum_{j=1}^N \hat{\gamma}_{jk} \left( \frac{(\mathbf{y}_j - \boldsymbol{\mu}_k)^T \Sigma_k^{-1}(\mathbf{y}_j - \boldsymbol{\mu}_k) }{\partial \boldsymbol{\mu}_k } \right) \\
				&= -\frac{1}{2} \sum_{j=1}^N \hat{\gamma}_{jk} \left( (\mathbf{y}_j - \boldsymbol{\mu}_k)^T (\Sigma_k^{-1})^{T} \frac{\partial (\mathbf{y}_j - \boldsymbol{\mu}_k) }{\partial \boldsymbol{\mu}_k}  +  (\mathbf{y}_j - \boldsymbol{\mu}_k)^T \Sigma_k^{-1} \frac{\partial (\mathbf{y}_j - \boldsymbol{\mu}_k) }{\partial \boldsymbol{\mu}_k} \right) \\
				&=  \frac{1}{2} \sum_{j=1}^N \hat{\gamma}_{jk} \left(  (\mathbf{y}_j - \boldsymbol{\mu}_k)^T \left( \Sigma_k^{-1} + (\Sigma_k^{-1})^{T} \right)   \right)
\end{aligned}
$$         

令$ \frac{\partial Q(\theta, \theta^{(i)})}{\partial  \boldsymbol{\mu}_k}=0 $可得          

$$
\begin{aligned}
\frac{1}{2} \sum_{j=1}^N \hat{\gamma}_{jk} \left(  (\mathbf{y}_j - \boldsymbol{\mu}_k)^T \left( \Sigma_k^{-1} + (\Sigma_k^{-1})^{T} \right)   \right) &=0 \\
			\sum_{j=1}^N \hat{\gamma}_{jk} (\mathbf{y}_j - \boldsymbol{\mu}_k)^T &=0 \\
			\boldsymbol{\mu}_k = \frac{\sum_{j=1}^N \hat{\gamma}_{jk} \mathbf{y}_j}{ \sum_{j=1}^N \hat{\gamma}_{jk}}
\end{aligned}
$$    

即      
$$ \boldsymbol{\mu}_k' = \frac{\sum_{j=1}^N \hat{\gamma}_{jk} \mathbf{y}_j}{ \sum_{j=1}^N \hat{\gamma}_{jk}} \quad k=1,2,\cdots,K \tag{5} $$         
同理可解得        
$$ \Sigma_k' = \frac{ \sum_{j=1}^N \hat{\gamma}_{jk} (\mathbf{y}_j - \boldsymbol{\mu}_k') (\mathbf{y}_j - \boldsymbol{\mu}_k')^T}{ \sum_{j=1}^N \hat{\gamma}_{jk} }  \tag{6} $$        
$$ \alpha_k' = \frac{\sum_{j=1}^N \hat{\gamma}_{jk}}{N} = \frac{n_k}{N} \tag{7} $$    


### 高斯混合模型聚类过程


<img src='../../../Other/img/GM.png' style="width:900px;height:600px">


1. 图(a).
   * 将样本集标记为绿色
   * 确定高斯混合成分个数为2
   * 初始化高斯混合分布的模型参数.将两个分模型的⼀个标准差位置的轮廓线分别用蓝色线圈和红色线圈表示,均值为线圈的质心处.
   
2. 图(b)
    * 对应第一次EM算法中的E步     
    * 根据式(3)计算每个数据点来自第$k$,$k=1,2$个分模型的概率.该数据来自第一个分模型的概率越大,则将该数据用更深的蓝色标记,若该数据来自第二个分模型的概率越大,则将该数据用更深的红色标记.若数据来自第一个分模型的概率和来自第二个分模型的概率接近,则颜色为紫色.
    
3. 图(c)
   * 对应第一次EM算法中的M步
   * 根据公式(6)更新每个分模型的均值
   * 根据公式(7)更新每个分模型的协方差矩阵(先更新均值,再由新的均值更新协方差均值)
   
4. 图(d),图(e),图(f)对应2次,5次,20次EM算法后的结果.图(f)中,算法接近收敛

5. 由图可知
    * $K$均值算法对数据点的聚类进行了“硬”分配,即每个数据点只属于唯⼀的聚类
    * 高斯混合模型算法基于后验概率分布,进行了⼀个“软”分配,即用概率向量表示数据点属于各聚类的概率